In [2]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib

app = Flask(__name__)

encoder = joblib.load(r"C:\Users\Admin\OneDrive\Desktop\SafeBite\Models\encoder.pkl")
model = joblib.load(r"C:\Users\Admin\OneDrive\Desktop\SafeBite\Models\best_knn_model.pkl")

@app.route("/")
def home():
    return jsonify({"message": "Welcome to the SafeBite Allergen Detection API"})

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.json
        input_data = pd.DataFrame([data])

        cat_cols = ['Food Product', 'Main Ingredient', 'Sweetener', 'Fat/Oil', 'Seasoning', 'Allergens']
        encoded_data = encoder.transform(input_data[cat_cols])
        input_data = pd.concat([input_data.drop(cat_cols, axis=1), pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())], axis=1)

        prediction = model.predict(input_data)
        prediction_proba = model.predict_proba(input_data)[:, 1]

        result = {
            "prediction": int(prediction[0]),  # 1: Contains allergens, 0: No allergens
            "confidence": float(prediction_proba[0])
        }

        return jsonify(result)

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2024 19:38:10] "POST /predict HTTP/1.1" 200 -
